In [1]:
import os
import pandas as pd
from IPython.display import display

In [2]:
external = pd.DataFrame()
path = os.getcwd().replace("notebooks","") + "data/raw/"
os.listdir(path)

['~$income.xlsx',
 'newest.json',
 'feature.json',
 '~$population.xlsx',
 'income.xlsx',
 'shapefile.zip',
 '.DS_Store',
 'lowest_prices.json',
 'covid_test.csv',
 'cemetery.csv',
 'shapefiles',
 'Median_Suburb_Quarterly_2020.xls',
 'Median_Suburb_Time_2021.xls',
 'Rental_Report_Quarterly_Median_2020.xlsx',
 '.gitkeep',
 'school_2021.csv',
 'functional.xlsx',
 'Rental_Report_Quarterly_Median_2021.xlsx',
 'school_2020.csv',
 'school_2022.csv',
 'Affordable_Lettings_2020.xlsx',
 'Gaming_Expenditure.xls',
 'Population_density_gaming_expenditures.xls',
 'highest_price.json',
 'datalink.txt',
 'Rental_Report_Quarterly.xlsx',
 'Government_Organisations.xlsx',
 'Sport_Recreational_Facilities_list.xlsx',
 'Affordable_Lettings_2021.xlsx',
 'Median_Suburb_Quarterly_2021.xlsx',
 'australian_postcodes.csv',
 'suburb.json',
 'ambulance.csv',
 'crime.xlsx',
 'population.xlsx']

In [3]:
postcode = pd.read_csv("../data/raw/australian_postcodes.csv")
postcode.columns

Index(['id', 'postcode', 'locality', 'state', 'long', 'lat', 'dc', 'type',
       'status', 'sa3', 'sa3name', 'sa4', 'sa4name', 'region', 'Lat_precise',
       'Long_precise', 'SA1_MAINCODE_2011', 'SA1_MAINCODE_2016',
       'SA2_MAINCODE_2016', 'SA2_NAME_2016', 'SA3_CODE_2016', 'SA3_NAME_2016',
       'SA4_CODE_2016', 'SA4_NAME_2016', 'RA_2011', 'RA_2016', 'MMM_2015',
       'MMM_2019', 'ced', 'altitude', 'chargezone', 'phn_code', 'phn_name',
       'lgaregion', 'electorate', 'electoraterating'],
      dtype='object')

In [4]:
external['postcode'] = postcode['postcode']
external['SA2_Code'] = postcode['SA2_MAINCODE_2016']
external['SA2_Name'] = postcode['SA2_NAME_2016']
external

,postcode,SA2_Code,SA2_Name
0,200,801051049.0,Acton
1,200,801051049.0,Acton
2,800,701011002.0,Darwin City
3,800,701011002.0,Darwin City
4,801,701011002.0,Darwin City
...,...,...,...
18437,9013,305011105.0,Brisbane City
18438,9015,305011105.0,Brisbane City
18439,9464,302031038.0,Northgate - Virginia
18440,9726,309101268.0,Bundall


In [5]:
population = pd.read_excel("../data/raw/population.xlsx",sheet_name="Table 1",header=7)
#population.columns
population = population.drop(columns=['S/T code', 'S/T name', 'GCCSA code', 'GCCSA name', 'SA4 code','SA4 name',
                                      'SA3 code', 'SA3 name',"Unnamed: 31","Unnamed: 34"])
population.head(5)

,SA2 code,SA2 name,no.,no..1,no..2,no..3,no..4,no..5,no..6,no..7,...,no..15,no..16,no..17,no..18,no..19,no..20,no..21,%,km2,persons/km2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,101021007.0,Braidwood,2760.0,2811.0,2835.0,2844.0,2847.0,2965.0,3102.0,3181.0,...,3950.0,4039.0,4140.0,4211.0,4273.0,4330.0,872.0,25.2,3418.4,1.3
2,101021008.0,Karabar,9129.0,9199.0,9263.0,9277.0,9209.0,9212.0,9033.0,8994.0,...,8531.0,8526.0,8507.0,8488.0,8519.0,8546.0,-546.0,-6.0,7.0,1223.9
3,101021009.0,Queanbeyan,9717.0,9513.0,9522.0,9400.0,9595.0,9682.0,9793.0,10074.0,...,11230.0,11355.0,11447.0,11450.0,11437.0,11370.0,655.0,6.1,4.8,2387.7
4,101021010.0,Queanbeyan - East,3925.0,4073.0,4219.0,4218.0,4187.0,4319.0,4459.0,4595.0,...,4970.0,5013.0,5072.0,5117.0,5077.0,5093.0,142.0,2.9,13.0,391.7


In [6]:
population.columns = ['SA2_Code', 'SA2 name','2001_population','2002_population','2003_population',
                      '2004_population','2005_population','2006_population', '2007_population','2008_population',
                      '2009_population', '2010_population', '2011_population', '2012_population', '2013_population',
                      '2014_population', '2015_population', '2016_population', '2017_population','2018_population',
                      '2019_population', '2020_population', '2021_population',"Population_Change_no",
                      "Population_Change_percent","Population_Area_km2","Population_density_2021_persons/km2"]
population = population.dropna()
display(population.tail(5))
population.shape

,SA2_Code,SA2 name,2001_population,2002_population,2003_population,2004_population,2005_population,2006_population,2007_population,2008_population,...,2016_population,2017_population,2018_population,2019_population,2020_population,2021_population,Population_Change_no,Population_Change_percent,Population_Area_km2,Population_density_2021_persons/km2
2450,801111141.0,Namadgi,12.0,11.0,11.0,10.0,10.0,9.0,15.0,22.0,...,39.0,45.0,50.0,56.0,61.0,67.0,26.0,63.4,1202.8,0.1
2451,901011001.0,Christmas Island,1442.0,1365.0,1337.0,1355.0,1380.0,1403.0,1569.0,1745.0,...,1903.0,1877.0,1849.0,1801.0,1752.0,1716.0,-451.0,-20.8,136.1,12.6
2452,901021002.0,Cocos (Keeling) Islands,600.0,568.0,558.0,573.0,588.0,590.0,575.0,568.0,...,546.0,569.0,571.0,599.0,605.0,602.0,41.0,7.3,13.7,43.9
2453,901031003.0,Jervis Bay,542.0,464.0,441.0,428.0,413.0,386.0,370.0,370.0,...,402.0,392.0,376.0,360.0,338.0,310.0,-79.0,-20.3,67.2,4.6
2454,901041004.0,Norfolk Island,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1757.0,1827.0,1920.0,1999.0,2096.0,2220.0,2220.0,0.0,38.7,57.4


(2454, 27)

In [7]:
#external = external.merge(population, left_on='SA2_Code', right_on='SA2 code')
external = pd.merge(external, population, on='SA2_Code', how='left').fillna(0)
external

,postcode,SA2_Code,SA2_Name,SA2 name,2001_population,2002_population,2003_population,2004_population,2005_population,2006_population,...,2016_population,2017_population,2018_population,2019_population,2020_population,2021_population,Population_Change_no,Population_Change_percent,Population_Area_km2,Population_density_2021_persons/km2
0,200,801051049.0,Acton,Acton,1488.0,1578.0,1677.0,1777.0,1859.0,1920.0,...,2126.0,2278.0,2450.0,2641.0,2778.0,2875.0,789.0,37.8,2.7,1050.3
1,200,801051049.0,Acton,Acton,1488.0,1578.0,1677.0,1777.0,1859.0,1920.0,...,2126.0,2278.0,2450.0,2641.0,2778.0,2875.0,789.0,37.8,2.7,1050.3
2,800,701011002.0,Darwin City,Darwin City,2140.0,2248.0,2320.0,2429.0,2661.0,2798.0,...,7591.0,7962.0,7856.0,7707.0,7709.0,7679.0,2626.0,52.0,3.2,2420.0
3,800,701011002.0,Darwin City,Darwin City,2140.0,2248.0,2320.0,2429.0,2661.0,2798.0,...,7591.0,7962.0,7856.0,7707.0,7709.0,7679.0,2626.0,52.0,3.2,2420.0
4,801,701011002.0,Darwin City,Darwin City,2140.0,2248.0,2320.0,2429.0,2661.0,2798.0,...,7591.0,7962.0,7856.0,7707.0,7709.0,7679.0,2626.0,52.0,3.2,2420.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18437,9013,305011105.0,Brisbane City,Brisbane City,2814.0,3206.0,4575.0,5448.0,6685.0,7455.0,...,10939.0,11661.0,12371.0,13122.0,14064.0,13779.0,4423.0,47.3,2.4,5823.8
18438,9015,305011105.0,Brisbane City,Brisbane City,2814.0,3206.0,4575.0,5448.0,6685.0,7455.0,...,10939.0,11661.0,12371.0,13122.0,14064.0,13779.0,4423.0,47.3,2.4,5823.8
18439,9464,302031038.0,Northgate - Virginia,Northgate - Virginia,5631.0,5639.0,5631.0,5690.0,5785.0,5858.0,...,6869.0,6900.0,7027.0,7172.0,7195.0,7169.0,575.0,8.7,6.1,1170.1
18440,9726,309101268.0,Bundall,Bundall,4188.0,4218.0,4235.0,4250.0,4273.0,4290.0,...,4704.0,4885.0,4957.0,4968.0,4986.0,4930.0,566.0,13.0,3.9,1263.5


In [11]:
income = pd.read_excel("../data/raw/income.xlsx",sheet_name="Table 1.4",header=6)
display(income.head(5))
income = income.drop(columns=['2014-15', '2015-16', '2016-17', '2017-18',
       '2018-19', '2014-15.1', '2015-16.1', '2016-17.1', '2017-18.1',
       '2018-19.1', '2014-15.2', '2015-16.2', '2016-17.2', '2017-18.2',
       '2018-19.2','Unnamed: 27'])
income.columns = ['SA2_Code', 'SA2 NAME','2014-15_median_income','2015-16_median_income','2016-17_median_income',
                  '2017-18_median_income','2018-19_median_income','2014-15_mean_income','2015-16_mean_income',
                  '2016-17_mean_income','2017-18_mean_income','2018-19_mean_income']
income.head(5)

,SA2,SA2 NAME,2014-15,2015-16,2016-17,2017-18,2018-19,2014-15.1,2015-16.1,2016-17.1,...,2015-16.3,2016-17.3,2017-18.3,2018-19.3,2014-15.4,2015-16.4,2016-17.4,2017-18.4,2018-19.4,Unnamed: 27
0,Australia,NaN,13102895,13358252,13678024,14069082,14425037,42,42,42,...,47692,48360,49805,51389,61036,61975,62594,64246,65953,NaN
1,New South Wales,NaN,4091347,4191542,4344997,4466941,4569650,42,42,42,...,48085,48700,50153,51818,62798,64493,65196,67200,68816,NaN
2,101021007,Braidwood,2133,2153,2262,2315,2361,50,50,50,...,39716,41288,42003,41593,47741,51074,51090,51594,51149,NaN
3,101021008,Karabar,4866,4937,4988,5059,5100,43,42,42,...,55870,57880,59295,61777,59548,61093,62535,63790,66335,NaN
4,101021009,Queanbeyan,6194,6419,6486,6595,6697,40,39,39,...,54999,55376,57848,60119,59310,60764,60958,62863,65874,NaN


,SA2_Code,SA2 NAME,2014-15_median_income,2015-16_median_income,2016-17_median_income,2017-18_median_income,2018-19_median_income,2014-15_mean_income,2015-16_mean_income,2016-17_mean_income,2017-18_mean_income,2018-19_mean_income
0,Australia,NaN,46854,47692,48360,49805,51389,61036,61975,62594,64246,65953
1,New South Wales,NaN,46879,48085,48700,50153,51818,62798,64493,65196,67200,68816
2,101021007,Braidwood,38093,39716,41288,42003,41593,47741,51074,51090,51594,51149
3,101021008,Karabar,54942,55870,57880,59295,61777,59548,61093,62535,63790,66335
4,101021009,Queanbeyan,54716,54999,55376,57848,60119,59310,60764,60958,62863,65874


In [12]:
external = pd.merge(external, income, on='SA2_Code', how='left').fillna(0)
external

,postcode,SA2_Code,SA2_Name,SA2 name,2001_population,2002_population,2003_population,2004_population,2005_population,2006_population,...,2014-15_median_income,2015-16_median_income,2016-17_median_income,2017-18_median_income,2018-19_median_income,2014-15_mean_income,2015-16_mean_income,2016-17_mean_income,2017-18_mean_income,2018-19_mean_income
0,200,801051049.0,Acton,Acton,1488.0,1578.0,1677.0,1777.0,1859.0,1920.0,...,10384,10332,8682,9306,10433,21494,21438,17656,16835,19479
1,200,801051049.0,Acton,Acton,1488.0,1578.0,1677.0,1777.0,1859.0,1920.0,...,10384,10332,8682,9306,10433,21494,21438,17656,16835,19479
2,800,701011002.0,Darwin City,Darwin City,2140.0,2248.0,2320.0,2429.0,2661.0,2798.0,...,51473,56408,56035,60937,57789,70661,78636,81685,87791,74682
3,800,701011002.0,Darwin City,Darwin City,2140.0,2248.0,2320.0,2429.0,2661.0,2798.0,...,51473,56408,56035,60937,57789,70661,78636,81685,87791,74682
4,801,701011002.0,Darwin City,Darwin City,2140.0,2248.0,2320.0,2429.0,2661.0,2798.0,...,51473,56408,56035,60937,57789,70661,78636,81685,87791,74682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18437,9013,305011105.0,Brisbane City,Brisbane City,2814.0,3206.0,4575.0,5448.0,6685.0,7455.0,...,35116,35942,33956,37501,39592,69218,65963,65904,69649,71479
18438,9015,305011105.0,Brisbane City,Brisbane City,2814.0,3206.0,4575.0,5448.0,6685.0,7455.0,...,35116,35942,33956,37501,39592,69218,65963,65904,69649,71479
18439,9464,302031038.0,Northgate - Virginia,Northgate - Virginia,5631.0,5639.0,5631.0,5690.0,5785.0,5858.0,...,54533,55445,55722,57498,59389,65369,66354,66705,68703,71806
18440,9726,309101268.0,Bundall,Bundall,4188.0,4218.0,4235.0,4250.0,4273.0,4290.0,...,43959,46370,44066,44184,48078,67719,71221,68178,71310,75576
